# Agent models and decision processes: key concepts with examples - reinforcement and imitation learning

In the field of artificial intelligence, when one wishes to create a system that observes the environment and performs some kind of action in response, one generally thinks of this as a **decision process**. This is of particular interest to researchers seeking to employ machine learning methods in robotics. When reading discourse surrounding this broad topic, it is quite common to come across terms such as "agents", "policies", "states", "actions", "reinforcement learning", "reward functions" and so forth. This notebook is intended to serve as a gentle introduction to these concepts and the more common theoretical frameworks associated with them. It also includes some simple worked examples of learning a policy for balancing a pole on a cart using reinforcement and imitation learning.

Bear in mind that is an extremely brief introduction to a *huge* field of active research, so there are many things not covered here, even basic ones.

## Overview

Broadly and informally, a **decision process** is a process where an **agent** takes an **observation** of the **state** of its **environment** and comes up with an **action** it should take using a **policy**. Now, that's a lot of big words all at once, so in more detail about each:

- **environment** -- a real or simulated "world" where objects exist in some configuration (its **state**). For the purposes of decision processes, it is usually assumed that this world has time which progresses in discrete steps;
- **state** -- a vector fully describing the configuration of everything that exists in the environment. This changes with time, and since time progresses in discrete steps, any environment can be described as a sequence of its states as they evolve over time $ (s_1, s_{2}, ... ) $. The state belongs to a set of all possible states $ s \in S $. The variables that make up this **state** vector can be discrete or continuous;
- **observation** -- a vector that includes some subset of the **state** variables. This is necessary because in most cases, the complete **state** of the system is impractical or impossible to observe. If the **observation** gives us enough information to fully determine the future **state** of everything we care about, it is generally treated the same as if it were the complete **state**;
- **agent** -- an entity, such as a robot controller, animal, human or AI subroutine managing a character in a video game, that exists in some sort of **environment**. What sets an agent apart from just any old object is that it can **observe** the **state** of its **environment** and possesses the means to alter it in some way (such as a means of locomotion to alter its own position, or actuators to move other things);
- **action** -- something the **agent** can do to influence the **state** of the environment. The **action** belongs to a set of all possible actions $ a \in A $ This is typically paired with a **transition function** which describes how the future state of the environment is altered when a given **action** is taken at a given **state**. The **transition function** can be deterministic $ T(s_t, a) = s_{t+1}$ or probabilistic $ T(s_t, s_{t+1}, a) = P(s_{t+1} \vert s_t, a) $. The variables that make up the **action** vector can be discrete or continuous, but in most classic reinforcement and imitation learning contexts we only deal with discrete actions;
- **policy** -- a function which dictates how the agent will respond to any given state in its environment. 

Now that we're familiar with the terms, we can turn to defining a specific **decision process**. There are many ways one could theoretically go about doing this, but in practice almost every paper related to reinforcement and imitation learning (until some very recent developments, see below) starts by laying down the definition of a **Markov Decision Process**, or MDP for short. There is some variability in how people choose to state it, but usually it will be something like

$$ MDP = (S, A, R, T) $$

where $ S $ represents the set of all possible states, $ A $ is the set of all possible actions, $ R: S \rightarrow \mathbb{R} $ is the **reward** function which tells us how well we did by ending up in this state and $ T: S \times A \rightarrow S \text{ or } P(S) $ is the (deterministic or probabilistic) transition function. There's quite a bit of wiggle room over the specifics of stating this (introducing objects such as $ \Gamma : S \rightarrow A $, a function that tells us what actions are available at which states) but the main takeaway here is that **the future state of the system depends only on the current state and the action taken by the agent**. This means there is no hysteresis -- we don't care about anything that happened before the current state or how we got here. Thus most of the theory dealing with this concept also assumes that any given agent will define the **policy** as $ \pi : S \rightarrow A \text{ or } P(A) $. 

The primary question the fields of **reinforcement and imitation learning** seek to answer is thus: *given a Markov Decision Process, what is the most effective way of discovering a policy function?* Both are discussed in more detail below, but the long and short of it is:

- In **reinforcement learning** we give our agent a random starting policy (typically in the form of some model template, an object which can approximate a wide range of possible functions -- such as a neural network) and let it loose upon the environment. This way we collect rewards which tell us how well we did. Using the rewards, we update the policy, then try again. Repeat until a useful agent pops out. The biggest advantage of this approach is that we don't have to know anything about how to actually achieve the goal, nor do we need a system that can. In theory, we only need to be able to define the reward function. In practice, however, there are several important drawbacks. First of all, this method is computationally extremely expensive, since the only way to learn is through interaction with the (typically simulated) environment. Secondly, many problems have extremely sparse rewards (meaning we can go a long time before seeing how well we did), which results in astronomically huge search spaces that agents have to explore before getting any useful feedback. Thus, the approach of
- **imtation learning** takes example state-action pairs produced by an **expert** and trains a policy that can reproduce these accurately. Obviously, this only works if we have an expert (such as a human demonstrator) to begin with. But provided we do, this lets us skip the incredibly expensive exploration phase of the process and go straight to learning. Furthermore, a policy obtained from a not very good expert can then be improved through reinforcement learning, and lots of research in the field is about precisely this -- using imitation learning to "kickstart" a reinforcement learning process.

So far, we've only discussed cases when the full state of the system is known and it follows the Markov Decision Process formalism. Things get somewhat murkier once we start dealing with incomplete observations -- it is possible that some hidden variables are not apparent from any given observation but can in fact be inferred from longer state histories. This is particularly often the case when dealing with agents which will have to operate in real rather than simulated environments -- such as robot controllers. Therefore the most recent state-of-the-art research, particularly regarding imitation learning, has been quite heavily slanted towards using sequence prediction models (such as transformer neural networks) which also keep track of past states, moving us away from the world of MDPs. However, in many cases methods grounded in the MDP are perfectly sufficient and those are the only ones used in examples below.

## The task - CartPole

The task used in all the examples below is a classic control problem called the **inverse pendulum**. Basically, you've got a stick on a cart. The stick is free to rotate, the cart is free to move left and right. The agent's got to keep the stick upright by sliding the cart underneath. If the stick goes below a certain angle, or the cart slides out of the picture, game over. For every timestep that the agent manages to survive it gets a $+1$ reward. This is great introductory illustration of various control-related concepts. In our case, we're using the *OpenAI gym* package to provide us with a ready-made simulation environment that accepts actions as inputs and outputs states at every timestep. Each action is a number -- either 0 or 1 -- that tells the cart to move left or right. Each state is a vector $ (x, v, \theta, \omega) $ where $x$ -- current position, $v$ -- current velocity, $\theta$ -- angle of the stick, $\omega$ -- rotational velocity of the stick. Visualization is handled by the *gym* package. All that's left for us to do is to write our own agent.

To get a picture of what this looks like with an untrained agent, below is an agent using the tried and true "take a random action every time" approach to success. *visualize_model* is a utility function that wraps the simulation environment for us, defined in *utils.py* in this repository.

In [ ]:
import numpy as np
from utils import visualize_model

class RandomAgent():
    
    def get_action(self, _):
        return np.random.randint(0,2)

model = RandomAgent()
visualize_model(model, notebook=True)

## Reinforcement learning

In reinforcement learning, the agent takes actions in its environment, collects a reward at every state it visits and uses this reward signal to optimize its policy. Simple enough to state, but an enormously broad problem in reality. A wide variety of different approaches have been studied, and it's hard to even begin trying to categorize them. But perhaps the most straightforward divide is that between value function and policy optimization methods.

**Policy optimization** methods optimize the policy directly. There are many ways to do this, but in principle the pattern is as follows: the policy is a model template (such as a neural network); this is used to collect reward information; the model is updated directly to take actions which result in greater reward. These methods typically struggle with noisy reward signals.

**Value function** methods try to estimate the value -- time-discounted sum of all future rewards -- of any given state, and use this estimate to select an action with the greatest expected return. The main reason to do things this way is that it lets us "average out" the noise in observed rewards. The Q-learning example below is a value function method.

In practice, combined methods also exist. The advantage actor-critic example below is a combined method, where an estimate of the value function is optimized alongside the policy to help stabilize and guide the optimization process.

### Q-learning

This is a classic value function estimation method, in use since the 1980s. In its most basic form, we keep a tensor (many-dimensional equivalent to a matrix or vector) $ Q $ with $ q_{s,a} \in Q $ corresponding to the expected value 

$$ \mathbb{E}\left[ R(T(s_t, a)) + \sum_{\tau={t+1}}^{\inf} \gamma^{\tau-t} R(s_{\tau}) \right]: s_{\tau} = T(s_{\tau-1}, \max_a \lbrace q_{s_{\tau-1},a} \rbrace) $$

Now, this equation may seem like quite the handful at first, and it's recursive to boot. Essentially, every cell contains the esimate of what we'd get if at state $s_t$ we took action $a$, but thereafter kept picking the action based on what we expect the maximum reward to be. The $ \gamma $ term is the **discount factor**, some number between 0 and 1 that lets us weigh rewards far in the future as less important than ones closer in time.

Of course, this assumes that we already have the Q-matrix filled out with all the values, which is a sort of chicken-and-egg problem. Thankfully, it's been proven that this kind of time-discounted value matrix will converge if we simply collect rewards and update the values at every time step. Specifically, the update rule used in the script is

$$ q_{s_t,a}^{i+1} = (1-\alpha)q_{s_t,a}^i + \alpha \sum_{\tau=t}^{t_{\max}}\gamma^{\tau-t} R(s_{\tau}) $$

$ s_t \in (s_1, s_2, ..., s_{t_{\max}}) $ is a state recorded in the training episode of length $t_{\max}$, $i$ is the current training iteration, $\alpha$ is a coefficient that determines how much we update the matrix with every training iteration. There is one major catch with using this method for the cartpole task -- the Q-matrix requires that our state values be discrete, or else the matrix would have to be infinitely large. Therefore, a large part of the implementation code is concerned with mapping the continuous position, angle and velocity observations to discrete values which can be used to index an array. A resolution has been picked arbitrarily to find a good compromise between performance and having a model that actually learns the task. The implementation code can be found in *QMatrix.py*.

To train the model, we first define a convenience function to update the learning rate with every training iteration. When starting out we can afford to make large updates, but as training progresses we don't want to keep throwing our model off an altering the induced state distribution (states actually visited by the agent when following a policy) too much.

In [ ]:
import math

def rate_schedule(ep, decay, min_rate = 0.01, initial = 1.0):
    return max( min_rate, min( initial, math.exp(-decay*ep) ) )


The actual training loop is rather simple compared to more sophisticated models. At the end of our process, we can choose to save our model to a file by simply pickling the QMatrix object.

In [ ]:

def train_matrix(model, env, save=True):

    for episode in range(4000):

        lr = rate_schedule(episode, 0.001)
        exploration_rate = rate_schedule(episode, 0.001)
        discount = 0.98

        done = False
        state_new, _ = env.reset()
        timestep = 0
        
        while not done and timestep < 2000:

            timestep += 1
            state_old = state_new
            action, _ = model.randomized_action(env.action_space, state_old, exploration_rate)

            state_new, reward, done, _, __ = env.step(action)
            expected_reward = reward + model.expected_next_reward(state_new, discount)
            model.update_reward(expected_reward, state_old, action, lr)

        if episode % 50 == 0:
            print(f"Training episode {episode} survived for {timestep} steps")     
    if save:
        model.save()

To call the training loop we need to instantiate the matrix and environment:

In [ ]:
from QMatrix import QMatrix
import gym

env = gym.make("CartPole-v1")

QSHAPE=(4,8,4,8) # how many discrete steps we split each continuous axis into
LIMITS=(
    (env.observation_space.low[0], env.observation_space.high[0]),
    (-10,10),
    (env.observation_space.low[2], env.observation_space.high[2]),
    (-5,5),
)
qmat = QMatrix(LIMITS, QSHAPE, env.action_space.n)

train_matrix(qmat, env, save=True)

After training (or if we choose to use a pre-trained model) we can load it from file and run it in the environment:

In [ ]:
from QMatrix import QMatrix
from utils import visualize_model

INNAME="qmatrix_models/QMatrix.pickle"

qmat = QMatrix.load(INNAME)
visualize_model(qmat, notebook=True)

As can be seen, the Q-matrix method is entirely sufficient for tasks with small state and action spaces like this, but in practice most problems will entail impossibly large matrices. That's why outside of toy examples there is very little use for this method, and more sophisticated approaches are required.

### Advantage Actor-Critic Neural Network

The advantage actor-critic method is a newer approach to reinforcement learning which utilizes a combination of value estimation and direct policy optimization.

In our case, a single neural network $\pi$ with parameters $\theta$ learns to predict an estimate of the value at the current state (similar to the value in the Q-matrix) as well as a set of action probabilities; more formally, 

$$ \pi_{\theta}(s_t) = \left( V_{\psi}(s_t), P_{\phi}(A) \right): \psi, \phi \subset \theta $$ 



The part of the model's output responsible for predicting the value is called the **critic** while the one giving us the action probabilities is called the **actor**. Just like before, during each training episode a sequence of states is recorded and the sum of future rewards is computed for each. This then gets used in the critic loss, which can be any general purpose loss function used in regression tasks

$$ R_{observed}(s_t) = \sum_{\tau=t}^{t_{\max}}\gamma^{\tau-t} R(s_{\tau}) $$

$$ \mathcal{L}_{critic}\left( V_{\psi}(s_t), R_{observed}(s_t) \right) =
 \mathcal{L}_{smoothL1loss}\left( V_{\psi}(s_t),  R_{observed}(s_t) \right) $$

This way, the critic learns to associate every state with a discounted future reward -- the value function part of this approach. The agent gets optimized using the actual reward $R_{observed}(s_t)$ and the critic's estimate of the value $V_{\psi}(s_t)$ to produce a term called **advantage** $ Adv(s_t) = R_{observed}(s_t) - V_{\psi}(s_t)$. The loss is then defined as 

$$ \mathcal{L}_{actor} = -\sum_{t=1}^{t_{\max}} Adv(s_t) \log\left[P_{\phi}(A \vert s_t) \right]  $$

The logarithmic probabilities $\log \left[P_{\phi}(A \vert s_t) \right] $ of the model  are flipped from negative to positive. Thus high probabilities are close to zero while low ones tend towards infinity). 
 The upshot of this is that

- when the model does much better than expected, adjusting the probability upward is given greater weight;
- when the model does much worse than expected, adjusting the probability downward is given greater weight;
- outputs which trend towards the expected value of the state are left alone as their weights approach 0.

The implementation code for this model can be found in *ActorCritic.py*. Specifically, we're using PyTorch to implement a deep neural network with one hidden layer, 128 hidden features and ReLU activation by default. This then feeds into a 2-feature action output (corresponding to *move left* and *move right*), and a single feature value output. The code for interfacing with the simulation environment is contained in the class itself, which is quite verbose and won't be considered in this notebook. Training can be invoked by

In [ ]:
import torch
from torch.optim import Adam
from ActorCritic import ActorCriticModel, SEED, ENV

OUTNAME="actor_critic_models/ActorCritic_adam"

def train_model(outname=OUTNAME):

    env = gym.make(ENV)
    env.reset(seed=SEED)

    dev = torch.device('cpu')
    model = ActorCriticModel(
        env.observation_space.shape[0],
        env.action_space.n,
        device=dev
    )
    model.train()

    opt = Adam(model.parameters(), lr=3e-2)

    for ep in range(1000):
        model.train_episode(env, opt, ep)
        if ep % 100 == 0:
            print(f"Saving model at epoch {ep}...")
            torch.save(model.state_dict(), f"{outname}_ep{ep}.pth")


while previously trained models can be loaded and visualized with

In [ ]:
import torch
from utils import visualize_model

INNAME="actor_critic_models/ActorCritic_adam_ep600.pth"

def evaluate_model(inname=INNAME):

    dev = torch.device('cpu')
    model = ActorCriticModel(device=dev)
    model.eval()
    model.load_state_dict(torch.load(inname))
    
    visualize_model(model)

## Imitation learning

In imitation learning, datasets consisting called "demonstrations" are collected by an agent already capable of accomplishing the task, called an "expert". Again, as with reinforcement learning, multiple broad approaches exist, and finding ways to enumerate them all is beyond the scope of this notebook. Perhaps the most useful distinction is between

- **behavioral cloning** -- demonstrations consist of simple state action pairs $(s_t, a_t)$. The policy $\pi_{\theta}(s_t)$ simply learns to approximate $a_t$ either as a classification (for discrete action spaces) or regression (for continuous ones) task. This is the simplest approach, but it struggles with so-called **distribution shift** - a divergence between states reached by the expert and those the cloned policy finds itself reaching, caused by compounding errors. Various mitigations have been studied, such as sampling the agent during training (if one is available), but in general this is a hard problem to overcome;
- **inverse reinforcement learning** -- first, an attempt is made to discover a reward function that could have produced the observed distribution of state-action pairs. Then reinforcement learning is used to optimize a policy for this reconstructed reward function. More advanced approaches try to sample the entire class of possible reward functions through generative-adversarial training methods.
- **time series behavioral cloning** -- the MDP formalism is discarded and longer sequences of past states are used to train general-purpose sequence predictors such as transformer or recurrent neural networks.

### Behavioral cloning

In this example, a neural network of the same architecture as the one used for the actor-critic is instead trained on demonstrations collected by either one of the previously trained agents. A simple categorical cross entropy loss function is used to train the model to predict the correct action corresponding to each state as in a typical classification task

$$ \mathcal{L}_{clone} (\pi_{\theta}(s_t), a_t) = \mathcal{L}_{crossentropy}(\pi_{\theta}(s_t), a_t)$$

The implementation can be found in *BehavioralClone.py*. For training, first it is necessary to collect demonstration data

In [1]:
import torch
from os.path import exists
from BehavioralClone import StateActionDataset

PROB_RANDOM=0.01

def collect_episode(env, model, exploration_rate=PROB_RANDOM):

    states = []
    actions = []

    state, _ = env.reset()
    done = False
    steps = 0

    while not done and steps < 1000:
        steps += 1
        action, was_random = model.randomized_action(env.action_space, torch.tensor(state), exploration_rate)
        if not was_random:
            states.append(state)
            actions.append(action)
        state, _, done, __, ___ = env.step(action)
    
    return torch.tensor(np.array(states)), torch.tensor(actions)


def collect_dataset(env, model, eps, name):

    if exists(name):
        print(f"Loading dataset from file {name}")
        return StateActionDataset.load_from_file(name)

    print(f"Collecting new dataset for {name}")

    state_list = []
    action_list = []

    for ep in range(eps):
        states, actions = collect_episode(env, model)
        state_list.append(states)
        action_list.append(actions)

        if ep % 50 == 0:
            print(f"Processed episode {ep} of {eps}")
    
    state_tensor = torch.concat(state_list)
    action_tensor = torch.concat(action_list)
    
    ds = StateActionDataset(state_tensor, action_tensor)
    print(f"Saving dataset to {name}")
    ds.save_to_file(name)

    return ds

With the dataset creation implemented, the rest is straightforward classificator training

In [ ]:
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from BehavioralClone import BehavioralClone, EPS_COLLECT_TRAIN, EPS_COLLECT_TEST, TRAIN_DATASET_FNAME, TEST_DATASET_FNAME, TRAIN_EPOCHS

def test_model(model, dataloader, device):

    correct_sum = torch.tensor(0).to(device)
    total_length = 0

    for state_batch, action_batch in dataloader:
        infer_batch = model.forward(state_batch)
        correct_sum += torch.where(infer_batch.argmax(dim=-1) == action_batch.to(device), 1, 0).sum()
        total_length += len(infer_batch)
    
    return (correct_sum / total_length).item()

def clone_agent(env, expert):

    train_ds = collect_dataset(env, expert, EPS_COLLECT_TRAIN, TRAIN_DATASET_FNAME)
    test_ds = collect_dataset(env, expert, EPS_COLLECT_TEST, TEST_DATASET_FNAME)

    train_dl = DataLoader(train_ds, batch_size=2048, shuffle=True)
    test_dl = DataLoader(test_ds, batch_size=64, shuffle=True)

    cpu = torch.device('cpu')
    device = torch.device('cuda') if torch.cuda.is_available() else cpu
    model = BehavioralClone(device, env.observation_space.shape[0], env.action_space.n)
    model.train()

    opt = Adam(model.parameters(), lr=1e-2)
    scd = MultiStepLR(opt, [1,2,5,10])
    losses = []
    for epoch in range(TRAIN_EPOCHS):
        print(f"\n\nStarting epoch {epoch} with lr {scd.get_last_lr()}")

        for i, (state_batch, action_batch) in enumerate(train_dl):
            infer_batch = model.forward(state_batch)
            loss = tf.cross_entropy(infer_batch, action_batch.to(device), reduction='sum')
            opt.zero_grad()
            loss.backward()
            opt.step()

            losses.append(loss)
            if i != 0 and i % 2000 == 0:
                print(f"Batch {i} mean loss {torch.stack(losses).mean()}")
                losses = []
        scd.step()
        correct_percent_train = test_model(model, train_dl, device) * 100
        correct_percent = test_model(model, test_dl, device) * 100
        print(f"Epoch {epoch} test accuracy {correct_percent}% train accuracy {correct_percent_train}%")
    
    return model

To create the datasets and clone an agent, run

In [ ]:
import gym

OUTNAME_BASE=f"bc_models/BehavioralClone_ds_{EPS_COLLECT_TRAIN}-{PROB_RANDOM}_epochs_{TRAIN_EPOCHS}_layers_"
env = gym.make(ENV)

expert = ActorCriticModel(env.observation_space.shape[0], env.action_space.n)
expert.load_state_dict(torch.load(INNAME))

cloned_model = clone_agent(env, expert)

torch.save(cloned_model.state_dict(), f"{OUTNAME_BASE}{cloned_model.name_modifier}.pth")

To evaluate a trained model, run

In [ ]:
from utils import visualize_model

EVAL_NAME="bc_models/BehavioralClone_4-128-2.pth"

cloned_model = BehavioralClone(torch.device('cuda'))
cloned_model.load_state_dict(torch.load(f"{EVAL_NAME}"))
cloned_model.eval()

visualize_model(cloned_model)